In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.decomposition import PCA 
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
import numpy as np
import seaborn as sns
from tqdm import tqdm
import matplotlib.pyplot as plt
from sklearn.cluster import MiniBatchKMeans
from sklearn.metrics.pairwise import pairwise_distances_argmin

import warnings
warnings.filterwarnings("ignore")

df_=pd.read_csv(r'data.csv')

df=df_.drop(['id','year','message_code','service','atm_status',
       'atm_manufacturer','atm_location','atm_streetname',
       'atm_street_number','atm_zipcode','weather_lat',"atm_id",
       'weather_lon','weather_city_id','weather_city_name','weather_description'],axis=1)


df['rain_3h']=df['rain_3h'].fillna(0)
df.message_text=df.message_text.fillna("None")

df.message_text = df.message_text.replace(["Suspected malfunction, no cash dispensed","Suspected malfunction, card retained"],"Suspected malfunction")
df.message_text = df.message_text.replace(["Timed-out taking card, card retained and no cash dispensed","Timed-out taking money"],"Timed-out")

In [2]:
def fuzzyday(x):
    a=0
    b=0
    c=0

    if x<4:
        a=1
    elif x<9:
        Y=x-4
        a = 1-Y*0.25
    elif x>8:
        a=0
        
    if x<4:
        b=0
    elif x<8:
        T = x-4
        b=T*0.25
    elif x<25:
        b=1
    elif x<29:
        T = x-25
        b=1-T*0.25
        T+=1
        
    if x<25:
        c=0
    elif x<29:
        L=x-25
        c=0.25*L
    elif x>28:
        c=1
    
    
    return round(a*10),round(b*10),round(c*10)

def fuzzyhour(x):
    a=0
    b=0
    c=0

    if x<6:
        a=1
    elif x<11:
        Y=x-5
        a = 1-Y*0.2
    elif x>10:
        a=0
        
    if x<6:
        b=0
    elif x<11:
        T = x-5
        b=T*0.2
    elif x<15:
        b=1
    elif x<21:
        T = x-15
        b=1-T*0.2
        
        
    if x<15:
        c=0
    elif x<20:
        L=x-15
        c=0.20*L
    elif x>20:
        c=1
    
    
    return round(a*10),round(b*10),round(c*10)

In [3]:
df["1day"]=1
df["2day"]=2
df["3day"]=3
df["1day"],df["2day"],df["3day"] = zip(*df.day.apply(lambda x:fuzzyday(x)))
df.drop("day",axis=1, inplace=True)

df["1hour"]=1
df["2hour"]=2
df["3hour"]=3
df["1hour"],df["2hour"],df["3hour"] = zip(*df.hour.apply(lambda x:fuzzyhour(x)))
df.drop("hour",axis=1, inplace=True)


dff = pd.get_dummies(df[["month","weekday","currency","card_type",'weather_main',"message_text"]],drop_first=True)

df = df.drop(["month","weekday","currency","card_type",'weather_main',"message_text"],axis=1)

df = pd.merge(df,dff,left_index=True,right_index=True)

MMS = MinMaxScaler()
MMS.fit(df)
df_norm = MMS.transform(df)
df_norm = pd.DataFrame(df_norm, columns=df.columns)
df_norm.head()

pca_model = PCA(n_components=0.95)
pca_model.fit(df_norm)
df_pca = pca_model.transform(df_norm)
df_pca = pd.DataFrame(df_pca)


pca_ratio=pca_model.explained_variance_ratio_
pca_ratio

array([0.09111237, 0.08091678, 0.06965594, 0.06097135, 0.05105725,
       0.0488295 , 0.04634851, 0.04567278, 0.04465698, 0.04452921,
       0.03984915, 0.03743889, 0.03676162, 0.03562685, 0.03452043,
       0.03234547, 0.02596047, 0.02200391, 0.02007065, 0.01652769,
       0.01496215, 0.01336286, 0.01223224, 0.00961662, 0.00820711,
       0.00806818])

In [4]:
distrToTal = []

for j in df_pca.columns:
    distr = []
    arange = []
    pas = 0
    sta = -1.25
    for i in range(2000): 
        lenn =len(df_pca[j][df_pca[j]<sta])
        #print(lenn)
        distr.append((lenn / 1250000) - pas)
        #print(distr)
        pas = np.sum(distr)
        #print(pas)
        arange.append(round(sta,2))
        sta +=0.02
        #print(sta)
        if sta>1.75:
            break
    distrToTal.append(distr)
distrToTal

[[0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0002792,
  0.00030399999999999996,
  0.0010976000000000002,
  0.0017247999999999999,
  0.0024663999999999997,
  0.004976,
  0.0057304,
  0.008339200000000001,
  0.010779999999999998,
  0.014397599999999997,
  0.017063199999999994,
  0.015577600000000011,
  0.0185448,
  0.023319199999999998,
  0.023823999999999998,
  0.022804799999999986,
  0.02099040000000002,
  0.018652000000000002,
  0.018167199999999994,
  0.016302399999999995,
  0.014941600000000027,
  0.0133664,
  0.01048479999999996,
  0.009937599999999991,
  0.00918800000000003,
  0.008921599999999974,
  0.009763200000000027,
  0.009975200000000017,
  0.010334399999999966,
  0.0110016,
  0.01249680000000003,
  0.011645599999999978,
  0.011114400000000024,
  0.00891759999999997,
  0.008447199999999988,
  0.007288000000000017,
  0.006720000000000004,
  0.006500799999999973,
  0.007685600000000015,
  0.009387199999999

In [5]:
def fitness(df):
    summy = []
    for j in df.columns:
        j = int(j)
        sc=0
        pas=0
        count=0
        summ=[]
        for i in arange:
            lenn =df[df[j]<i].shape[0]
            pers = (lenn / df.shape[0]  ) - pas
            pas += pers
            summ.append(np.abs(pers - distrToTal[j][count]))
            count+=1
        summy.append(np.sum(summ) * pca_ratio[j])
        
    return np.sum(summy)

In [6]:
kol = []
p = -1
indexs = np.zeros((26,625010))
for i in tqdm(df_pca.columns):
#     print("=====================================================")
    a=0
    listeo= []
    ghelgheli = []
    ali=0
    q = 0
    p+=1
    for j in arange:
        a+=1
#     b+=1
        if a>149:
            c=10000
        else:
            c = arange[a]
    #         print(b)
#         print("***",j,c,"***")
        dfff = df_pca[(df_pca[i]>j) & (df_pca[i]<c)]
        po = round(len(dfff)/2)
        poq = po+q           
#         print(po,q,poq,len(dfff))

        indexs[p][q:poq] = np.random.choice(dfff.index , po)
        q+=po
#         ali +=round(len(dfff)/10)
#         porya=round(len(dfff)/10)
#         if ali>125000:
#             porya = porya - (ali - 125000)
#             ali = ali - (ali - 125000)
#         ghelgheli.append(porya)
        
# #         np.random.choice(dfff.index , round(len(dfff)/10), replace=False)
#     #         print(a)
#         print(len(dfff))
#         print(ghelgheli)
#     while (np.sum(ghelgheli)<125000):
# #         ghelgheli.append(np.random.randint(0,1250000))
#         ghelgheli.append(1)
#     print(indexs)
#     print(np.sum(ghelgheli))
    
#     kol.append(listeo)
print(indexs)

100%|██████████████████████████████████████████████████████████████████████████████████| 26/26 [01:11<00:00,  2.74s/it]


[[ 83740.  84730.  84844. ...      0.      0.      0.]
 [286145. 286006. 286140. ...      0.      0.      0.]
 [121032. 121121.  42154. ...      0.      0.      0.]
 ...
 [572072. 559650. 193565. ...      0.      0.      0.]
 [144075.  65721. 507213. ...      0.      0.      0.]
 [708354. 994944. 734878. ...      0.      0.      0.]]


In [14]:
m=indexs[0][indexs[0]!=0]
for i in range(6):
    m=m[np.isin(m,indexs[i])]

len(m)

5940

In [31]:
print("Our sampeling score: " + str(fitness(df_pca.loc[m])))
rnd=np.random.choice(1250000,len(m))
print("Random sampeling score: " + str(fitness(df_pca.loc[rnd])))

Our sampeling score: 0.02750287085997959
Random sampeling score: 0.025648357277323175


In [20]:
df_pca.iloc[m].to_csv(r"common_sample.csv")

In [23]:
fitness(1-df_pca.iloc[m])

1.4650136833986533